In [1]:
run = '3fk8mrp3'

In [2]:
from functools import partial
from pathlib import Path
import hydra
from hydra import initialize, compose
from typing import Dict, List
import os

import omegaconf
import pytorch_lightning
from nn_core.common import PROJECT_ROOT
import dataclasses
from typing import List
from nn_core.callbacks import NNTemplateCore
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO
from datasets import Dataset, DatasetDict
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from torch.utils.data import DataLoader
from pytorch_lightning import Callback
from torch.utils.data import DataLoader
from tqdm import tqdm
import dataclasses
from typing import Union
import random
from hydra.utils import instantiate
from pydoc import locate
from nn_core.serialization import load_model
from typing import List
from nn_core.callbacks import NNTemplateCore
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from pytorch_lightning import Callback
import logging
import json
import re 
from omegaconf import OmegaConf

pylogger = logging.getLogger(__name__)

/root/miniconda3/envs/ccmm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/ccmm/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/envs/ccmm/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
# load 100 epochs 

storage_path = Path(f'../storage/cycle-consistent-model-merging/{run}')
ckpt_path = storage_path / "checkpoints"

In [4]:
import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="mlp")

hydra.initialize()

In [5]:
# cfg = compose(config_name="mlp", overrides=[])
with open(storage_path / "config.yaml", "r") as f:
    conf_string = f.read()
    cfg = OmegaConf.create(conf_string)

In [6]:
def load_model_ckpt(model_path: str) -> pl.LightningModule:

    model_class = locate("ccmm.pl_modules.pl_module.MyLightningModule")
    model = load_model(model_class, checkpoint_path=Path(model_path))
    model.eval().cuda()
    return model


In [7]:
def load_and_sort_checkpoints(checkpoint_dir):
    # Get all files in the directory
    checkpoint_files = os.listdir(checkpoint_dir)

    # Filter only .ckpt files
    checkpoint_files = [file for file in checkpoint_files if file.endswith('.ckpt.zip')]

    # Sort files based on epoch number
    checkpoint_files.sort(key=lambda x: int(re.search(r'epoch=(\d+)', x).group(1)))

    
    checkpoints = [load_model_ckpt(os.path.join(checkpoint_dir, file)) for file in checkpoint_files]
    return checkpoints

In [8]:
checkpoints = load_and_sort_checkpoints(ckpt_path)

/data/cycle-consistent-model-merging/src/ccmm/pl_modules/pl_module.py:142: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")


In [9]:
models_by_epoch = checkpoints

In [10]:
import torch

K_start = 0
K_end = 10

all_params = []
for epoch_model in models_by_epoch[K_start:K_end]:
    params = epoch_model.state_dict()
    # flatten and concatenate all the parameters in a single vector
    params = torch.cat([p.flatten() for p in params.values()])
    all_params.append(params)

all_params = torch.stack(all_params)   

In [11]:
# (K, num_params_per_model)
all_params.shape

torch.Size([9, 2232586])

In [12]:
all_differences = []
K = K_end - K_start
for i in range(1, K):
    all_differences.append(all_params[i] - all_params[i-1])

all_differences = torch.stack(all_differences)

In [13]:
# shape (num_params_per_model, K-1)
all_differences = all_differences.transpose(1, 0)
all_differences.shape

torch.Size([2232586, 8])

In [14]:
Q, R = torch.linalg.qr(all_differences)

In [15]:
R.shape[0]

8

In [16]:
d = torch.linalg.solve(R.conj().t() @ R, -torch.ones_like(R[0]))
gamma = d / d.sum()

In [17]:
proj_weights = (all_params[:-1].t() * gamma).sum(1)

In [18]:
# copy model_by_epoch[0] to new_model
import copy
new_model = copy.deepcopy(models_by_epoch[0])
# get the original shape of the parameter layers
shapes = [p.shape for p in models_by_epoch[0].state_dict().values()]
# reshape proj_weights into the original shape
proj_weights = proj_weights.split([p.numel() for p in models_by_epoch[0].state_dict().values()])
proj_weights = [p.reshape(s) for p, s in zip(proj_weights, shapes)]
# assign the weights to the model
for p, w in zip(new_model.state_dict().values(), proj_weights):
    p.copy_(w)


In [19]:
len(models_by_epoch)

100

In [20]:
# define a pl trainer to use for evaluation
datamodule = instantiate(cfg.nn.data, _recursive_=False)
trainer_old = pl.Trainer(**cfg.train.trainer)
trainer_old.test(models_by_epoch[K_end], datamodule=datamodule)
trainer_new = pl.Trainer(**cfg.train.trainer)
trainer_new.test(new_model, datamodule=datamodule)

Files already downloaded and verified


Files already downloaded and verified


/root/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(


2023-12-30 14:13:00 INFO     GPU available: True (cuda), used: True      ]8;id=830469;file:///root/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=369679;file:///root/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1751\1751]8;;\

                    INFO     TPU available: False, using: 0 TPU cores    ]8;id=787860;file:///root/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=481541;file:///root/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#1754\1754]8;;\

...

                    INFO     `Trainer(val_check_interval=1.0)` was       ]8;id=27767;file:///root/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py\pytorch_lightning.utilities.rank_zero]8;;\:]8;id=487469;file:///root/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py#2830\2830]8;;\
                             configured so validation will run at the                                              
                             end of the training epoch..                                                           

                    INFO     LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]       ]8;id=667634;file:///root/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py\pytorch_lightning.accelerators.cuda]8;;\:]8;id=335966;file:///root/miniconda3/envs/ccmm/lib/python3.9/site-packages/pytorch_lightning/accelerators/cuda.py#57\57]8;;\

Testing DataLoader 0: 100%|██████████| 20/20 [00:00<00:00, 26.29it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         acc/test          │    0.5461999773979187     │
│         loss/test         │    1.6293829679489136     │
└───────────────────────────┴───────────────────────────┘

[{'loss/test': 1.6293829679489136, 'acc/test': 0.5461999773979187}]